In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import numpy as np
import cv2
import random
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data.dataset import random_split

In [2]:
def print_params(model):
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(name, param.data)

In [3]:
def layers_connexion(model,input_size):
    Layers = 1 #input layer
    Connexions = []
    Nodes = [input_size]
    
    for name, param in model.named_parameters():
        if param.requires_grad:
            if "bias" in name:
                Layers += 1
                Nodes.append(len(param.detach().numpy()))
            else:
                a = param.detach().numpy()
            #Replace weight value by binary number
                a = np.select( [a != 0.0, a== 0.0], [1,0],default = a)
                a = np.array(a)
                Connexions.append(a)
    return(Layers, Connexions, Nodes)

In [4]:
class Net_Task2(torch.nn.Module):
    def __init__(self,input_dimension, output_dimension):
        super(Net_Task2, self).__init__()
        self.fc1 = nn.Linear(in_features = input_dimension, out_features = 5)
        self.fc2 = nn.Linear(in_features = 5, out_features = 5)
        self.fc3 = nn.Linear(in_features = 5, out_features = 5)
        #self.fc4 = nn.Linear(in_features = 5, out_features = 5)
        #self.fc5 = nn.Linear(in_features = 5,  out_features = output_dimension)
        self.fc4 = nn.Linear(in_features = 5,  out_features = output_dimension)

        #self.mask2 = torch.tensor([[1,0],[1,0],[0,1],[0,1]])
        #self.mask3 = torch.tensor([[1,1,0,0],[0,0,1,1]]) 


    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        #x = F.relu(self.fc4(x))
        #x = self.fc5(x)
        x = self.fc4(x)
        return torch.sigmoid(x)

class Net_Task1(torch.nn.Module):
    def __init__(self,input_dimension, output_dimension):
        super(Net_Task1, self).__init__()
        self.fc1 = nn.Linear(in_features = input_dimension, out_features = 1)
        self.fc2 = nn.Linear(in_features = 1, out_features = 2)
        self.fc3 = nn.Linear(in_features = 2,  out_features = output_dimension)


    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return torch.sigmoid(x)



In [5]:
model = Net_Task2(input_dimension = 5, output_dimension = 1)
print('\n The model parameters before the update are: \n')
print_params(model)


 The model parameters before the update are: 

fc1.weight tensor([[ 0.3493, -0.1160, -0.4156, -0.2876, -0.4431],
        [ 0.2046,  0.1510,  0.0583, -0.2949, -0.1399],
        [-0.1876, -0.0539,  0.0821, -0.1443, -0.3343],
        [ 0.2357, -0.0684,  0.2576,  0.3220, -0.0623],
        [ 0.3518, -0.0751, -0.0800,  0.1829, -0.2625]])
fc1.bias tensor([-0.0763,  0.0087, -0.1736,  0.2453,  0.0244])
fc2.weight tensor([[ 0.0301,  0.0340,  0.4053, -0.1280, -0.1441],
        [ 0.0829,  0.0380, -0.1956,  0.0228,  0.4180],
        [-0.1766, -0.2767,  0.3092, -0.3832,  0.2796],
        [-0.2450, -0.3009, -0.4135,  0.3988,  0.2293],
        [-0.1781,  0.2571,  0.2955,  0.1245, -0.0179]])
fc2.bias tensor([-0.0993, -0.1250, -0.1507, -0.2781,  0.1909])
fc3.weight tensor([[ 0.1410,  0.2814, -0.3597,  0.0520, -0.2148],
        [-0.1506, -0.3182, -0.3427,  0.2924,  0.3265],
        [-0.0106, -0.1717,  0.2426,  0.2002, -0.2602],
        [-0.0886, -0.3128, -0.1377, -0.0938,  0.4262],
        [-0.3179, -0.

In [6]:
class Network_Visualization:
    def __init__(self,display,magnification,Model,input_size):
        self.layers,self.connexions,self.nodes = layers_connexion(model,input_size)
        # Set whether the environment should be displayed after every step
        self.display = display
        # Set the magnification factor of the display
        self.magnification = magnification
        # Create an image which will be used to display the environment
        self.image = np.zeros([int(self.magnification), int(self.magnification), 3], dtype=np.uint8)
        # Set the width and height of the environment
        self.width = 5
        self.height = 5
        self.rayon = 20
            
        # Init y coordinates of each layers
        self.b = np.linspace(4* self.rayon,
                        self.magnification-4*self.rayon,
                        self.layers)
        # Init coordinates of each nodes, i//self.layers allow to have the same y coordinates for every nodes of the same layer
        self.nodes_coordinates()

    def nodes_coordinates(self):
        self.Nodes_coordinates = []
        N_max = np.argmax(self.nodes)  
        a = np.linspace(4* self.rayon, self.magnification-4*self.rayon, self.nodes[N_max])
        inter_space = a[1] - a[0]

        for i in range(self.layers):
            width_bound = self.nodes[i]*2+self.rayon + (self.nodes[i] - 1)*inter_space
            lower_bound = int(np.floor((self.magnification- width_bound)/2))
            upper_bound = int(np.floor((self.magnification + width_bound)/2))
            print("lower_bound",lower_bound)
            print(max([4* self.rayon,lower_bound]))
            a = np.linspace(np.max([4* self.rayon,lower_bound]),
                            np.min([self.magnification-4*self.rayon,upper_bound]),
                            self.nodes[i])
            coordinates = [[int(a[j]),int(self.b[i])] for j in range(len(a))]
            self.Nodes_coordinates.append(coordinates)    
    
    def draw(self):
        # Create the background image
        window_top_left = (0, 0)
        window_bottom_right = (self.magnification , self.magnification )
        cv2.rectangle(self.image, window_top_left, window_bottom_right, (255, 255, 255), thickness=cv2.FILLED)
        #Draw the Nodes
        
        for i in range(self.layers):
            for j in range(len(self.Nodes_coordinates[i])):
                cv2.circle(self.image,center= self.Nodes_coordinates[i][j], 
                           radius = self.rayon, 
                           color =(139,139,0), 
                           thickness = 2 )
        #Draw the connexions
        for i in reversed(range(1, self.layers )):
            g = self.connexions[i-1].shape
            for j in range(g[0]):
                for k in range(g[1]):
                    if self.connexions[i-1][j][k] != 0:
                      
                        cv2.line(self.image, 
                                 pt1 = np.array(self.Nodes_coordinates[i-1][k]) + [0,self.rayon], 
                                 pt2 = np.array(self.Nodes_coordinates[i][j]) - [0,self.rayon],
                                 color =  (238,229,142) , 
                                 thickness = 2)

        cv2.imshow("Neural Network", self.image)
        # This line is necessary to give time for the image to be rendered on the screen
        cv2.waitKey(1)
        

In [7]:
display_on = True
visu = Network_Visualization(display = display_on,magnification=800,Model = model,input_size=5)

# Determine the time at which training will stop, i.e. in 10 minutes (600 seconds) time
start_time = time.time() 
end_time = start_time + 30
visu.draw()
#while time.time() < end_time:
# Optionally, show the environment
 #   if display_on:
  #      visu.draw()

lower_bound 65
80
lower_bound 65
80
lower_bound 65
80
lower_bound 65
80
lower_bound 389
389


In [8]:

magnification = 800
height = 5
width = 5
image = np.zeros([int(magnification), int(magnification), 3], dtype=np.uint8)
window_top_left = (0, 0)
window_bottom_right = (magnification*height, magnification*width)
cv2.rectangle(image, window_top_left, window_bottom_right, (0, 238, 229), thickness=cv2.FILLED)
cv2.imshow("Environment6",image)
